In [1]:
from showdown.engine.helpers import normalize_name, calculate_stats
from poke_env.player.battle_order import ForfeitBattleOrder
import json
import numpy as np
import constants
from showdown.engine.find_state_instructions import get_all_state_instructions, get_state_instructions_from_move
from showdown.engine.objects import State, StateMutator, Pokemon, Side
from collections import defaultdict
from copy import deepcopy
import random
import config, pickle
from poke_env.data import TYPE_CHART
from showdown.engine.damage_calculator import calculate_damage
from poke_env.player_configuration import _create_player_configuration_from_player
from poke_env.player_configuration import PlayerConfiguration
from poke_env.server_configuration import LocalhostServerConfiguration, ShowdownServerConfiguration
from data.helpers import get_standard_battle_sets
# -*- coding: utf-8 -*-
import asyncio, nest_asyncio
import time

from poke_env.player.player import Player
from poke_env.player.random_player import RandomPlayer
from poke_env.player.baselines import SimpleHeuristicsPlayer
np.set_printoptions(precision=3, suppress=True)
with open('pokedex.json', 'r') as fp:
    pokedex = json.load(fp)

with open('moves.json', 'r') as fp:
    all_moves = json.load(fp)
    
def reformat_evs(evs):
    return [int(x) for x in '252,0,4,0,252,0'.split(',')]
def rename_baseStats(baseStats):
    return {'hp': baseStats['hp'], 'attack': baseStats['atk'], 'defense': baseStats['def'], 
           'special-attack': baseStats['spa'], 'special-defense': baseStats['spd'], 'speed': baseStats['spe']}
def max_pp(pp):
    return np.floor(pp * 1.6)

class HeuristicsPlayer():
    def __init__(self):
        self.ANTI_HAZARDS_MOVES = {"rapidspin", "defog"}
        self.SPEED_TIER_COEFICIENT = 0.1
        self.HP_FRACTION_COEFICIENT = 0.4
        self.SWITCH_OUT_MATCHUP_THRESHOLD = -2
        self.entry_hazards = ['stealthrock', 'spikes','stickyweb','toxicspikes']
        self.hazard_dict = {'stealthrock': 'stealth_rock', 'spikes': 'spikes', 'toxicspikes': 'toxic_spikes', 'stickyweb' : 'sticky_web'}
        
    def custom_damage_multiplier(self, mon, opponent):
        result = []
        for my_type in mon.types:
            mod = 1.0
            for your_type in opponent.types:
                mod *= TYPE_CHART[my_type.upper()][your_type.upper()]
            result.append(mod)
        return max(result)
        
    def _estimate_matchup(self, mon, opponent):
        score = self.custom_damage_multiplier(mon, opponent)
        score -= self.custom_damage_multiplier(opponent, mon)
        if mon.speed > opponent.speed:
            score += self.SPEED_TIER_COEFICIENT
        elif opponent.speed > mon.speed:
            score -= self.SPEED_TIER_COEFICIENT

        score += mon.hp / mon.maxhp * self.HP_FRACTION_COEFICIENT
        score -= opponent.hp/opponent.maxhp * self.HP_FRACTION_COEFICIENT

        return score
    
    def _should_switch_out(self, state):
        active = state.self.active
        opponent = state.opponent.active
        # If there is a decent switch in...
        if active.hp < 1:
            return True
        if len([m for m in state.self.reserve.values() if self._estimate_matchup(m, opponent) > 0]) > 0:
            # ...and a 'good' reason to switch out
            if active.defense_boost <= -3 or active.special_defense_boost <= -3:
                return True
            if (
                active.attack_boost <= -3
                and active.attack >= active.special_attack
            ):
                return True
            if (
                active.special_attack <= -3
                and active.attack <= active.special_attack
            ):
                return True
            if (
                self._estimate_matchup(active, opponent)< self.SWITCH_OUT_MATCHUP_THRESHOLD
            ):
                return True
        return False
    
    def choose_move(self, state):
        # Main mons shortcuts
        active = state.self.active
        opponent = state.opponent.active
        battle_moves = [x for x in state.get_self_options(False) if x[:7] != 'switch ']
        switch_moves = [x for x in state.get_self_options(False) if x[:7] == 'switch ']
#         print(switch_moves)
#         print(active)
#         print([x.hp for x in state.self.reserve.values()])
        if opponent.hp < 1 and active.hp >= 1:
            return 'splash'
        # Rough estimation of damage ratio
        physical_ratio = active.attack / active.defense
        special_ratio = active.special_attack / active.special_defense

        if len(battle_moves) > 0 and (
            not self._should_switch_out(state) or not len(switch_moves) > 0
        ):
            n_remaining_mons = len(
                [m for m in state.self.reserve.values() if m.hp < 1]
            )
            n_opp_remaining_mons = n_remaining_mons

            # Entry hazard...
            for move in battle_moves:
                # ...setup
                if (
                    n_opp_remaining_mons >= 3
                    and move in self.entry_hazards
                    and state.opponent.side_conditions[self.hazard_dict[move]] > 0
                ):
                    return move

                # ...removal
                elif (
                    any(v > 0 for v in state.self.side_conditions.values())
                    and move in self.ANTI_HAZARDS_MOVES
                    and n_remaining_mons >= 2
                ):
                    return move
            damages = []
            for move in battle_moves:
                damage = calculate_damage(state, 'self', move, 'splash', calc_type='average')
                if damage:
                    damages.append(damage[0][0])
                else:
                    damages.append(0)
            if max(damages) > 0 and opponent.hp == 1 or max(damages) > 1:
                return battle_moves[np.argmax(damages)]
        if switch_moves:
            switch_options = []
            for switch in switch_moves:
                mon = switch[7:]
                switch_options.append(self._estimate_matchup(state.self.reserve[mon], opponent))
            return switch_moves[np.argmax(switch_options)]
        
        return state.get_self_options(False)[np.random.randint(len(state.get_self_options(False)))]
    
class PokeBandit():
    def __init__(self, mutator, shp = None):
        self.mutator = mutator
        self.choices = self.mutator.state.get_self_options(False)
        self.n_arms = len(self.choices)
        self.q = np.zeros(self.n_arms)
        self.n = np.zeros(self.n_arms)
        self.shp = shp
        self.state_log = []
        self.move_log = []
        self.SPEED_TIER_COEFICIENT = 0.1
        self.HP_FRACTION_COEFICIENT = 0.4
        self.turn_avg = np.zeros(self.n_arms)
    
    def select_arm(self):
        new_arm = np.where(self.n == 0)[0]
        if len(new_arm) > 0:
            return new_arm[0]
        
        ucb1 = self.q/self.n + np.sqrt(2 * np.log(np.sum(self.n)) / self.n)
        return np.argmax(ucb1)
    
    def select_opp_arm(self, shp_select):
        
        new_arm = np.where(self.n == 0)[0]
        if len(new_arm) > 0:
            return new_arm[0]
        
        if np.random.random() < 0.1:
            return shp_select      
        ucb1 = self.q/self.n + np.sqrt(2 * np.log(np.sum(self.n)) / self.n)
        return np.argmax(ucb1)   
    
    
    def create_opponent(self):
        opp_mutator = deepcopy(self.mutator)
        #opp_mutator = pickle.loads(pickle.dumps(self.mutator))
        opp_mutator.state = self.flip_state(opp_mutator.state)
        self.opponent = PokeBandit(mutator=opp_mutator, shp=self.shp)
    
    def flip_state(self, state):
        state_flipped = state.deepcopy()
        tmp = state_flipped.self
        state_flipped.self = state_flipped.opponent
        state_flipped.opponent = tmp
        return state_flipped
    
    def playout_policy(self, state):
        if np.random.random() < 0.2:
            return self.shp.choose_move(state)
        return self.legal_move(state)
    
    def opponent_bandit_policy(self, state):
        if np.random.random() < 0.8:
            return self.shp.choose_move(state)
        return self.legal_move(state)
    
    def estimate_matchup(self, mon, opponent):
        score = self.custom_damage_multiplier(mon, opponent)
        score -= self.custom_damage_multiplier(opponent, mon)
        if mon.speed > opponent.speed:
            score += self.SPEED_TIER_COEFICIENT
        elif opponent.speed > mon.speed:
            score -= self.SPEED_TIER_COEFICIENT

        score += mon.hp / mon.maxhp * self.HP_FRACTION_COEFICIENT
        score -= opponent.hp/opponent.maxhp * self.HP_FRACTION_COEFICIENT

        return score
    
    def custom_damage_multiplier(self, mon, opponent):
        result = []
        for my_type in mon.types:
            mod = 1.0
            for your_type in opponent.types:
                mod *= TYPE_CHART[my_type.upper()][your_type.upper()]
            result.append(mod)
        return max(result)
    
    def legal_move(self, state):
        # Main mons shortcuts
        active = state.self.active
        opponent = state.opponent.active
        battle_moves = [x for x in state.get_self_options(False) if x[:7] != 'switch ']
        switch_moves = [x for x in state.get_self_options(False) if x[:7] == 'switch ']
        if opponent.hp < 1 and active.hp >= 1:
            return 'splash'
        if active.hp < 1:
            switch_options = []
            for switch in switch_moves:
                mon = switch[7:]
                switch_options.append(self.estimate_matchup(state.self.reserve[mon], opponent))
            return switch_moves[np.argmax(switch_options)]
        
        return state.get_self_options(False)[np.random.randint(len(state.get_self_options(False)))]
    
    
    def move(self, new_battle, my_move, opp_move):
        transpose_instructions = get_all_state_instructions(new_battle, my_move,opp_move)
        percentage_list = [x.percentage for x in transpose_instructions]
        instruction_list = [x.instructions for x in transpose_instructions]
#         print(transpose_instructions)
        instruction = random.choices(instruction_list, weights=percentage_list, k=1)[0]
#         print('')
#         print(instruction)
        new_battle.apply(instruction)
        
        if 'switch ' !=  my_move[:7]:
            for x in new_battle.state.self.active.moves:
                if x['id'] == my_move:
                    if new_battle.state.opponent.active.ability != 'Pressure':
                        x['current_pp'] -= 1
                    else:
                        x['current_pp'] -= 2
                    if x['current_pp'] <= 0:
                        x['disabled'] = True
        else:
            for x in new_battle.state.self.reserve.values():
                x.attack_boost = 0
                x.defense_boost = 0
                x.special_attack_boost = 0
                x.special_defense_boost = 0
                x.speed_boost = 0
                x.evasion_boost = 0
                x.accuracy_boost = 0
                if hasattr(x, 'volatileStatus'):
                    x.volatileStatus = set()

        if 'switch ' !=  opp_move[:7]:
            for x in new_battle.state.opponent.active.moves:
                if x['id'] == opp_move:
                    if new_battle.state.self.active.ability != 'Pressure':
                        x['current_pp'] -= 1
                    else:
                        x['current_pp'] -= 2
                    if x['current_pp'] <= 0:
                        x['disabled'] = True

        else:
            for x in new_battle.state.self.reserve.values():
                x.attack_boost = 0
                x.defense_boost = 0
                x.special_attack_boost = 0
                x.special_defense_boost = 0
                x.speed_boost = 0                
                x.evasion_boost = 0
                x.accuracy_boost = 0  
                if hasattr(x, 'volatileStatus'):
                    x.volatileStatus = set()
                    
        return new_battle
    
    def pull_arm(self, selected_arm, opponent_arm):
        new_battle = deepcopy(self.mutator)
        #new_battle = pickle.loads(pickle.dumps(self.mutator))
        new_state = new_battle.state
        flipped_state = self.flip_state(new_state)
        my_move = self.choices[selected_arm]
        
        opp_move = self.opponent.choices[opponent_arm]

#         print(new_battle.state.self.active)
#         print('')
#         print(new_battle.state.opponent.active)     
#         print('')
#         print(new_battle.state.field, new_battle.state.terrain_count)
#         print('')
#         print(my_move, opp_move)   
        new_battle = self.move(new_battle, my_move, opp_move)
#         print('-----------------------------------------------') 
        turns = 1
        while new_battle.state.battle_is_finished() == False:
#             print(new_battle.state.self.active)
#             print('')
#             print(new_battle.state.opponent.active)     
#             print('')
#             print(new_battle.state.field, new_battle.state.terrain_count, new_battle.state.weather_count)
#             print('')
            flipped_state = self.flip_state(new_battle.state)
            my_move = self.playout_policy(new_battle.state)
            opp_move = self.opponent.playout_policy(flipped_state)
            new_battle = self.move(new_battle, my_move, opp_move)
            turns += 1
#             print(my_move, opp_move)   
#             print('-----------------------------------------------') 
        
        reward = new_battle.state.battle_is_finished()
        if reward == -1:
            reward = 0
        return reward, turns
    
    def play_game(self, simulations_number=None, total_simulation_seconds=None):
        """
        Parameters
        ----------
        simulations_number : int
            number of simulations performed to get the best action
        total_simulation_seconds : float
            Amount of time the algorithm has to run. Specified in seconds
        Returns
        -------
        """
        
        self.create_opponent()
        if simulations_number is None :
            assert(total_simulation_seconds is not None)
            end_time = time.time() + total_simulation_seconds
            opp_state = self.flip_state(self.mutator.state)
            opp_options = opp_state.get_self_options(False)
            shp_select = opp_options.index(self.shp.choose_move(opp_state))
            while time.time() < end_time:
                chosen_arm = self.select_arm()
                opponent_arm = self.opponent.select_opp_arm(shp_select)
                reward, turns = self.pull_arm(chosen_arm, opponent_arm)
                self.n[chosen_arm] += 1
                self.q[chosen_arm] += reward
                self.opponent.n[opponent_arm] += 1
                self.opponent.q[opponent_arm] += 1 - reward
                self.turn_avg[chosen_arm] += turns
                
                
                if np.sum(self.n) > 200:
                    if all(self.q/self.n > 0.9):
                        print('ez')
                        break
                    error_bound = (self.q * (1 - self.q / self.n) ** 2 + 
                                   (self.n-self.q) * (self.q/self.n) ** 2)/self.n/np.sqrt(self.n)

                    upper_bound = self.q/self.n + 1.95*error_bound
                    lower_bound = self.q/self.n - 1.95*error_bound
                    if sum(max(lower_bound) < upper_bound) == 1:
                        print('confident')
                        break


        else :
            for _ in range(0, simulations_number):            
                chosen_arm = self.select_arm()
                opponent_arm = self.opponent.select_opp_arm()
                reward, turns = self.pull_arm(chosen_arm, opponent_arm)
                self.n[chosen_arm] += 1
                self.q[chosen_arm] += reward
                self.opponent.n[opponent_arm] += 1
                self.opponent.q[opponent_arm] += 1 - reward
                
        self.turn_avg /= self.n
    def best_action(self, simulations_number=None, total_simulation_seconds=None):
        self.play_game(simulations_number, total_simulation_seconds)
        if all(self.q/self.n > 0.9) and np.random.random() < 0.3:
            return self.shp.choose_move(self.mutator.state)
        return self.choices[np.argmax(self.q/self.n)]

In [19]:
mut_ex = []
config.damage_calc_type = 'average'
logs = []

class BanditPlayer(Player):
    
    async def _handle_message(self, message):
        rtn = await RandomPlayer._handle_message(self, message)
        msgs.append(message)
        return rtn

    def create_my_party(self, battle):
        my_party_pokes = []
        volatile_status = set()
        if battle.active_pokemon.effects != {}:
            for x in battle.active_pokemon.effects:
                if x == 'LEECH_SEED':
                    x = 'leechseed'
                if normalize_name(x.name) in ["flinch","confusion","leechseed","substitute","taunt","roost",
                                         "protect","banefulbunker","spikyshield", "dynamax","partiallytrapped",'transform']:
                    volatile_status.add(normalize_name(x.name))
        trapped_counter = 0
        substitute_hp = 0
        if 'partiallytrapped' in volatile_status:
            trapped_counter = 3
            
        poke_moves = []
        for move in battle.available_moves:
            disabled = (move.current_pp == 0)
            poke_moves.append({'id': normalize_name(move.id), 'disabled': disabled, 'current_pp': move.current_pp})
        
        types = []
        for x in list(battle.active_pokemon.types):
            if x != None:
                types.append(normalize_name(x.name))
        if battle.active_pokemon.status != None:
            status = normalize_name(battle.active_pokemon.status.name)
        else:
            status = None
            
        if 'substitute' in volatile_status:
            substitute_hp = battle.active_pokemon.max_hp / 8
            
        my_party_pokes.append(Pokemon(
            identifier=normalize_name(battle.active_pokemon.species),
            level=battle.active_pokemon.level,
            types=types,
            hp = battle.active_pokemon.current_hp,
            maxhp = battle.active_pokemon.max_hp,
            ability = battle.active_pokemon.ability,
            item = battle.active_pokemon.item,
            attack=battle.active_pokemon.stats['atk'],
            defense=battle.active_pokemon.stats['def'],
            special_attack=battle.active_pokemon.stats['spa'],
            special_defense=battle.active_pokemon.stats['spd'],
            speed=battle.active_pokemon.stats['spe'],
            attack_boost=battle.active_pokemon.boosts['atk'],
            defense_boost=battle.active_pokemon.boosts['def'],
            special_attack_boost=battle.active_pokemon.boosts['spa'],
            special_defense_boost=battle.active_pokemon.boosts['spd'],
            speed_boost=battle.active_pokemon.boosts['spe'],
            accuracy_boost=battle.active_pokemon.boosts['accuracy'],
            evasion_boost=battle.active_pokemon.boosts['evasion'],
            status=status,
            volatile_status = volatile_status,
            moves= poke_moves,
            substitute_hp = substitute_hp,
            trapped_counter = trapped_counter
        ))

        for mon in battle.available_switches:
            poke_moves = []
            for move in mon.moves.values():
                disabled = (move.current_pp == 0)
                poke_moves.append({'id': normalize_name(move.id), 'disabled': disabled, 'current_pp': move.current_pp})
            types = []
            for x in list(mon.types):
                if x != None:
                    types.append(normalize_name(x.name))


            my_party_pokes.append(Pokemon(
                identifier = normalize_name(mon.species),
                level=mon.level,
                types=types,
                hp = mon.current_hp,
                maxhp = mon.max_hp,
                ability = mon.ability,
                item = mon.item,
                attack=mon.stats['atk'],
                defense=mon.stats['def'],
                special_attack=mon.stats['spa'],
                special_defense=mon.stats['spd'],
                speed=mon.stats['spe'],
                attack_boost=mon.boosts['atk'],
                defense_boost=mon.boosts['def'],
                special_attack_boost=mon.boosts['spa'],
                special_defense_boost=mon.boosts['spd'],
                speed_boost=mon.boosts['spe'],
                accuracy_boost=mon.boosts['accuracy'],
                evasion_boost=mon.boosts['evasion'],
                status=None,
                volatile_status = set(),
                moves= poke_moves
            ))
        return my_party_pokes
    
    def create_opp_party(self, battle):   
        opp_party_pokes = []
        opp_team_species = set([normalize_name(x.species) for x in self.opponent_preview])
        for mon in battle.opponent_team.values():
            volatile_status = set()
            if mon.effects != {}:
                for x in mon.effects:
                    if x == 'LEECH_SEED':
                        x = 'leechseed'
                    if normalize_name(x.name) in ["flinch","confusion","leechseed","substitute","taunt","roost",
                                             "protect","banefulbunker","spikyshield", "dynamax","partiallytrapped",'transform']:
                        volatile_status.add(normalize_name(x.name))
            substitute_hp = 0
            trapped_counter = 0
            if 'partiallytrapped' in volatile_status:
                trapped_counter = 3
            opp_team_species.remove(normalize_name(mon.species))
            poke_moves = []
            for move in mon.moves.values():
                disabled = (move.current_pp == 0)
                poke_moves.append({'id': normalize_name(move.id), 'disabled': disabled, 'current_pp': move.current_pp})

            if len(poke_moves) < 4:
                for move in self.sets[normalize_name(mon.species)]['moves']:
                    if move[0] not in [x['id'] for x in poke_moves]:
                        poke_moves.append({'id': normalize_name(move[0]), 'disabled': False, 'current_pp': max_pp(all_moves[normalize_name(move[0])]['pp'])})
                    if len(poke_moves) == 4:
                        break
            if mon.status != None:
                status = normalize_name(mon.status.name)
            else:
                status = None
            types = [normalize_name(x) for x in pokedex[normalize_name(mon.species)]['types']]
            stats = calculate_stats(rename_baseStats(pokedex[normalize_name(mon.species)]['baseStats']),
                    100, ivs=(31,) * 6, evs=reformat_evs(self.sets[mon.species]['spreads'][0][1]), 
                                    nature=self.sets[mon.species]['spreads'][0][0])
            ability = mon.ability
            if ability == None:
                ability = self.sets[normalize_name(mon.species)]['abilities'][0][0]
            item = mon.item
            if item == None:
                item = self.sets[normalize_name(mon.species)]['items'][0][0]
            
            if mon.boosts == None:
                boosts = [0, 0, 0, 0, 0, 0, 0]
            else:
                boosts = [mon.boosts['atk'], mon.boosts['def'], mon.boosts['spa'],
                          mon.boosts['spd'], mon.boosts['spe'], mon.boosts['accuracy'], mon.boosts['evasion']]

            if 'substitute' in volatile_status:
                substitute_hp = stats['hp'] / 8
            
            opp_party_pokes.append(Pokemon(
                identifier = normalize_name(mon.species),
                level=mon.level,
                types=types,
                hp = np.floor(mon.current_hp / 100 * stats['hp']),
                maxhp = stats['hp'],
                ability = ability,
                item = item,
                attack=stats['attack'],
                defense=stats['defense'],
                special_attack=stats['special-attack'],
                special_defense=stats['special-defense'],
                speed=stats['speed'],
                attack_boost=boosts[0],
                defense_boost=boosts[1],
                special_attack_boost=boosts[2],
                special_defense_boost=boosts[3],
                speed_boost=boosts[4],
                accuracy_boost=boosts[5],
                evasion_boost=boosts[6],
                status=status,
                volatile_status = volatile_status,
                moves= poke_moves,
                substitute_hp = substitute_hp,
                trapped_counter=trapped_counter
            ))
        for mon in opp_team_species:
            poke_moves = []
            for move in self.sets[mon]['moves']:
                if move[0] not in [x['id'] for x in poke_moves]:
                    poke_moves.append({'id': normalize_name(move[0]), 'disabled': False, 'current_pp': max_pp(all_moves[normalize_name(move[0])]['pp'])})
                if len(poke_moves) == 4:
                    break
            types = [normalize_name(x) for x in pokedex[mon]['types']]
            stats = calculate_stats(rename_baseStats(pokedex[mon]['baseStats']),
                    100, ivs=(31,) * 6, evs=reformat_evs(self.sets[mon]['spreads'][0][1]), 
                                    nature=self.sets[mon]['spreads'][0][0])
            
            ability = self.sets[normalize_name(mon)]['abilities'][0][0]
            item = self.sets[normalize_name(mon)]['items'][0][0]
            
            boosts = [0, 0, 0, 0, 0, 0, 0]
            
            opp_party_pokes.append(Pokemon(
                identifier = mon,
                level=100,
                types=types,
                hp = stats['hp'],
                maxhp = stats['hp'],
                ability = ability,
                item = item,
                attack=stats['attack'],
                defense=stats['defense'],
                special_attack=stats['special-attack'],
                special_defense=stats['special-defense'],
                speed=stats['speed'],
                attack_boost=boosts[0],
                defense_boost=boosts[1],
                special_attack_boost=boosts[2],
                special_defense_boost=boosts[3],
                speed_boost=boosts[4],
                accuracy_boost=boosts[5],
                evasion_boost=boosts[6],
                status=None,
                volatile_status = set(),
                moves= poke_moves
            ))
            
        return opp_party_pokes
    
    def choose_move(self, battle):
        global mut_ex
        global logs
        print("TIME TO CHOOSE A MOVE!")
        # If the player can attack, it will
        my_team = self.create_my_party(battle)
        opp_team = self.create_opp_party(battle)
        side_conditions= defaultdict(int)
        my_conds = [0, 0, 0]
        if len(battle.side_conditions) > 0:
            for i in battle.side_conditions.keys():
                if i.name == 'STEALTH_ROCK':
                    side_conditions['stealthrock'] = 1
                elif i.name == 'TOXIC_SPIKES':
                    side_conditions['toxicspikes'] = 1
                elif i.name == 'SPIKES':
                    side_conditions['spikes'] = 1
                elif i.name == 'STICKY_WEB':
                    side_conditions['stickyweb'] = 1
                elif i.name == 'TAILWIND':
                    side_conditions['tailwind'] = 1
                elif i.name == 'LIGHT_SCREEN':
                    side_conditions['lightscreen'] = 1
                    my_conds[0] = 3
                elif i.name == 'REFLECT':
                    side_conditions['reflect'] = 1
                    my_conds[1] = 3
                elif i.name == 'AURORA_VEIL':
                    side_conditions['aurora_veil'] = 1
                    my_conds[2] = 3
        opp_side_conditions= defaultdict(int)
        opp_conds = [0, 0, 0]
        if len(battle.opponent_side_conditions) > 0:
            for i in battle.opponent_side_conditions.keys():
                if i.name == 'STEALTH_ROCK':
                    opp_side_conditions['stealth_rock'] = 1
                elif i.name == 'TOXIC_SPIKES':
                    opp_side_conditions['toxic_spikes'] = 1
                elif i.name == 'SPIKES':
                    opp_side_conditions['spikes'] = 1
                elif i.name == 'STICKY_WEB':
                    opp_side_conditions['sticky_web'] = 1
                elif i.name == 'TAILWIND':
                    opp_side_conditions['tailwind'] = 1   
                elif i.name == 'LIGHT_SCREEN':
                    opp_side_conditions['lightscreen'] = 1
                    opp_conds[0] = 3
                elif i.name == 'REFLECT':
                    opp_side_conditions['reflect'] = 1
                    opp_conds[1] = 3
                elif i.name == 'AURORA_VEIL':
                    opp_side_conditions['aurora_veil'] = 1
                    opp_conds[2] = 3
        if len(my_team) == 1:
            my_reserve = {}
        else:
            my_reserve = my_team[1:]
            
        opp_reserve = {}
        if len(opp_team) != 1:
            for x in opp_team:
                if x.id == normalize_name(battle.opponent_active_pokemon.species):
                    opp_active = x
                else:
                    opp_reserve[x.id] = x     
        
        side1 = Side(
            active=my_team[0],
            reserve={x.id: x for x in my_reserve},
            wish=(0, 0),
            side_conditions=side_conditions,
            ls_count=my_conds[0],
            ref_count=my_conds[1],
            av_count=my_conds[2]
        )

        side2 = Side(
            active=opp_active,
            reserve=opp_reserve,
            wish=(0, 0),
            side_conditions=opp_side_conditions,
            ls_count=opp_conds[0],
            ref_count=opp_conds[1],
            av_count=opp_conds[2]
        )
        weather = None
        weather_count = 0
        if battle.weather != {}:
            for weath in battle.weather:
                weather = weath.name
            weather_count = 3
            
        field = None
        terrain_count = 0
        if battle.fields != {}:
            for fi in battle.fields:
                if fi.name == "ELECTRIC_TERRAIN":
                    field = 'electricterrain'
                elif fi.name == "GRASSY_TERRAIN":
                    field = 'grassyterrain'
                elif fi.name == 'PSYCHIC_TERRAIN':
                    field = 'psychicterrain'
                elif fi.name == 'MISTY_TERRAIN':
                    fi.name = 'mistyterrain'
            terrain_count = 3           
        state = State(
            user=side1,
            opponent=side2,
            weather=weather,
            field=None,
            trick_room=False,
            weather_count=weather_count,
            terrain_count=terrain_count
        )
        mut_ex = StateMutator(state)
#        print(mut_ex, )
        has_defog = False
        has_sr = False
        for move in battle.available_moves:
            if normalize_name(move.id) == 'defog':
                has_defog=True
            elif normalize_name(move.id) == 'stealthrock':
                has_sr = True
        if len(battle.opponent_side_conditions) > 0:
            opp_hazards = False
            my_hazards = False
            opp_screens = False            
            for i in battle.opponent_side_conditions.keys():
                if i.name == 'STEALTH_ROCK':
                    opp_hazards = True
                elif i.name == 'LIGHT_SCREEN' or i.name == 'REFLECT' or i.name=='AURORA_VEIL':
                    opp_screens = True
                    break
            if has_defog:
                for i in battle.side_conditions.keys():
                    if i.name == 'STEALTH_ROCK' or i.name == 'TOXIC_SPIKES' or i.name=='SPIKES' or i.name == 'STICKY_WEB':
                        my_hazards = True

                if my_hazards == False and opp_screens == False:
                    for move in mut_ex.state.self.active.moves:
                        if move['id'] == 'defog':
                            move['disabled'] = True
            if has_sr:
                if opp_hazards == True:
                    for move in mut_ex.state.self.active.moves:
                        if move['id'] == 'stealthrock':
                            move['disabled'] = True                    
                            


        logs.append(deepcopy(mut_ex))
        
        if len(battle.available_moves) == 0:
            mon_performance = {}
            for mon in battle.available_switches:
                mon_performance[mon] = self.teampreview_performance(mon, battle.opponent_active_pokemon)
            ordered_mons = sorted(mon_performance, key=lambda k: -mon_performance[k])
            return self.create_order(ordered_mons[0])
        return ''
        bandit = PokeBandit(mut_ex, shp = HeuristicsPlayer())
#         action = bandit.best_action(total_simulation_seconds=15)
        try:
            action = bandit.best_action(total_simulation_seconds=20)
        except Exception as e:
            print("Something went wrong!")
            action = bandit.legal_move(state)
        print(action)
#         print(bandit.q)
#         print(bandit.n)
        print("Sum: ", np.sum(bandit.n))
        print("Win %: ", bandit.q/bandit.n)
        print("Play % ", bandit.n/np.sum(bandit.n))
        print('Formula: ', bandit.q/bandit.n + bandit.n/np.sum(bandit.n))
        print('Turns: ', bandit.turn_avg)
        print(bandit.choices)
        
#         if np.max(bandit.q/bandit.n) < 0.1:
#             return ForfeitBattleOrder()
        if action[:7] == 'switch ':
            mon = action[7:]
            for x in battle.available_switches:
                if mon == normalize_name(x.species):
                    return self.create_order(x)

        else:
            for move in battle.available_moves:
                if action == normalize_name(move.id):
                    return self.create_order(move) 
        print("what")
        return "what"
        
        
        
    def teampreview(self, battle):
        print("TIME FOR TEAM PREVIEW")
        if hasattr(self, 'my_team'):
            print('hi')
            print(self.my_team)
            return self.my_team
        mon_performance = {}
        self.opponent_preview = battle.opponent_team.values()
        opponent_mons = [normalize_name(x.species) for x in battle.opponent_team.values()]
        self.sets = get_standard_battle_sets('gen8ou',opponent_mons)
        if len(self.sets) != len(self.opponent_preview):
            leftovers = []
            for mon in opponent_mons:
                if mon not in self.sets:
                    leftovers.append(mon)
            leftover_sets = get_standard_battle_sets('all',leftovers)
            for mon in leftover_sets:
                self.sets[mon] = leftover_sets[mon]        
        
        # For each of our pokemons
        for i, mon in enumerate(battle.team.values()):
            # We store their average performance against the opponent team
            mon_performance[i] = np.mean(
                [
                    self.teampreview_performance(mon, opp)
                    for opp in battle.opponent_team.values()
                ]
            )

        # We sort our mons by performance
        ordered_mons = sorted(mon_performance, key=lambda k: -mon_performance[k])

        # We start with the one we consider best overall
        # We use i + 1 as python indexes start from 0
        #  but showdown's indexes start from 1
        self.my_team = "/team " + "".join([str(i + 1) for i in ordered_mons])
        print(self.my_team)
        return self.my_team


    def teampreview_performance(self, mon_a, mon_b):
        # We evaluate the performance on mon_a against mon_b as its type advantage
        a_on_b = b_on_a = -np.inf
        for type_ in mon_a.types:
            if type_:
                a_on_b = max(a_on_b, type_.damage_multiplier(*mon_b.types))
        # We do the same for mon_b over mon_a
        for type_ in mon_b.types:
            if type_:
                b_on_a = max(b_on_a, type_.damage_multiplier(*mon_a.types))
        # Our performance metric is the different between the two
        return a_on_b - b_on_a  

async def main():
    start = time.time()
    with open('team1.txt', 'r') as fp:
        team_1 = fp.read()
    with open('team2.txt', 'r') as fp:
        team_2 = fp.read()
    
    # We create two players.
    shp_player = SimpleHeuristicsPlayer(
        battle_format="gen8ou",
        team=team_2,
#         player_configuration=PlayerConfiguration("mingu600", "#Dyklsu03"),
#         server_configuration=LocalhostServerConfiguration,
        max_concurrent_battles=1,
        log_level=40
    )

    bandit_player = BanditPlayer(
        battle_format="gen8ou",
        team=team_2,
        max_concurrent_battles=1,
#         player_configuration=PlayerConfiguration("BanditPlayer", "pororo"),
#         server_configuration=LocalhostServerConfiguration,
        log_level=40
    )
    
#    await bandit_player.send_challenges("mingu600", n_challenges=1)

# #     # Accepting one challenge from any user
#     await bandit_player.accept_challenges(None, 1)
    
#     await bandit_player.ladder(1)
#     await bandit_player.accept_challenges(None, 1)
    
#     for battle in bandit_player.battles.values():
#         print(battle.rating, battle.opponent_rating)
    
    # Now, let's evaluate our player
    await shp_player.battle_against(bandit_player, n_battles = 1)

#     print(
#         "bandit player won %d / 100 battles"
#         % bandit_player.n_won_battles
#     )


nest_asyncio.apply()
try:
    asyncio.get_event_loop().run_until_complete(main())
except ERROR:
    pass

Task exception was never retrieved
future: <Task finished coro=<BanditPlayer._handle_message() done, defined at <ipython-input-13-592dd3e7058d>:7> exception=NameError("name 'msgs' is not defined",)>
Traceback (most recent call last):
  File "/Users/mingukim/.pyenv/versions/3.6.13/lib/python3.6/asyncio/tasks.py", line 180, in _step
    result = coro.send(None)
  File "<ipython-input-13-592dd3e7058d>", line 9, in _handle_message
    msgs.append(message)
NameError: name 'msgs' is not defined
Task exception was never retrieved
future: <Task finished coro=<BanditPlayer._handle_message() done, defined at <ipython-input-13-592dd3e7058d>:7> exception=AttributeError("'str' object has no attribute 'message'",)>
Traceback (most recent call last):
  File "/Users/mingukim/.pyenv/versions/3.6.13/lib/python3.6/asyncio/tasks.py", line 180, in _step
    result = coro.send(None)
  File "<ipython-input-13-592dd3e7058d>", line 8, in _handle_message
    rtn = await RandomPlayer._handle_message(self, messag

TIME FOR TEAM PREVIEW


Task exception was never retrieved
future: <Task finished coro=<BanditPlayer._handle_message() done, defined at <ipython-input-19-c98f30ba7a38>:7> exception=NameError("name 'msgs' is not defined",)>
Traceback (most recent call last):
  File "/Users/mingukim/.pyenv/versions/3.6.13/lib/python3.6/asyncio/tasks.py", line 180, in _step
    result = coro.send(None)
  File "<ipython-input-19-c98f30ba7a38>", line 9, in _handle_message
    msgs.append(message)
NameError: name 'msgs' is not defined
Task exception was never retrieved
future: <Task finished coro=<BanditPlayer._handle_message() done, defined at <ipython-input-19-c98f30ba7a38>:7> exception=NameError("name 'msgs' is not defined",)>
Traceback (most recent call last):
  File "/Users/mingukim/.pyenv/versions/3.6.13/lib/python3.6/asyncio/tasks.py", line 180, in _step
    result = coro.send(None)
  File "<ipython-input-19-c98f30ba7a38>", line 9, in _handle_message
    msgs.append(message)
NameError: name 'msgs' is not defined
Task excepti

/team 324561
TIME TO CHOOSE A MOVE!


NameError: name 'ERROR' is not defined

In [3]:
len(logs)

5

In [10]:
# turn = 4
# print(logs[turn].state.self.active)
# print('')
# print(logs[turn].state.opponent.active)
# print('')
# print(logs[turn].state.opponent.side_conditions)
# print('')
# print(logs[turn].state.self.reserve)
# print('')
# print(logs[turn].state.opponent.reserve)

In [20]:
mut_ex_copy = deepcopy(mut_ex)

In [21]:
mut_ex_copy.state.self.active

{'id': 'clefable', 'level': 100, 'types': ['fairy'], 'hp': 394, 'maxhp': 394, 'ability': 'magicguard', 'item': 'leftovers', 'attack': 158, 'defense': 203, 'special-attack': 226, 'special-defense': 276, 'speed': 164, 'nature': 'serious', 'evs': (85, 85, 85, 85, 85, 85), 'attack_boost': 0, 'defense_boost': 0, 'special_attack_boost': 0, 'special_defense_boost': 0, 'speed_boost': 0, 'accuracy_boost': 0, 'evasion_boost': 0, 'status': None, 'volatileStatus': [], 'moves': [{'id': 'moonblast', 'disabled': False, 'current_pp': 15}, {'id': 'knockoff', 'disabled': False, 'current_pp': 20}, {'id': 'softboiled', 'disabled': False, 'current_pp': 10}, {'id': 'thunderwave', 'disabled': False, 'current_pp': 20}], 'part_trapped_counter': 0, 'substitute_hp': 0}

In [22]:
mut_ex_copy.state.opponent.active

{'id': 'zapdos', 'level': 100, 'types': ['electric', 'flying'], 'hp': 384.0, 'maxhp': 384, 'ability': 'static', 'item': 'unknown_item', 'attack': 196, 'defense': 207, 'special-attack': 286, 'special-defense': 279, 'speed': 259, 'nature': 'serious', 'evs': (85, 85, 85, 85, 85, 85), 'attack_boost': 0, 'defense_boost': 0, 'special_attack_boost': 0, 'special_defense_boost': 0, 'speed_boost': 0, 'accuracy_boost': 0, 'evasion_boost': 0, 'status': None, 'volatileStatus': [], 'moves': [{'id': 'roost', 'disabled': False, 'current_pp': 16.0}, {'id': 'hurricane', 'disabled': False, 'current_pp': 16.0}, {'id': 'defog', 'disabled': False, 'current_pp': 24.0}, {'id': 'heatwave', 'disabled': False, 'current_pp': 16.0}], 'part_trapped_counter': 0, 'substitute_hp': 0}

In [12]:
mut_ex_copy

In [26]:
class HeuristicsPlayer():
    def __init__(self):
        self.ANTI_HAZARDS_MOVES = {"rapidspin", "defog"}
        self.SPEED_TIER_COEFICIENT = 0.1
        self.HP_FRACTION_COEFICIENT = 0.4
        self.SWITCH_OUT_MATCHUP_THRESHOLD = -2
        self.entry_hazards = ['stealthrock', 'spikes','stickyweb','toxicspikes']
        self.hazard_dict = {'stealthrock': 'stealth_rock', 'spikes': 'spikes', 'toxicspikes': 'toxic_spikes', 'stickyweb' : 'sticky_web'}
        
    def custom_damage_multiplier(self, mon, opponent):
        result = []
        for my_type in mon.types:
            mod = 1.0
            for your_type in opponent.types:
                mod *= TYPE_CHART[my_type.upper()][your_type.upper()]
            result.append(mod)
        return max(result)
        
    def _estimate_matchup(self, mon, opponent):
        score = self.custom_damage_multiplier(mon, opponent)
        score -= self.custom_damage_multiplier(opponent, mon)
        if mon.speed > opponent.speed:
            score += self.SPEED_TIER_COEFICIENT
        elif opponent.speed > mon.speed:
            score -= self.SPEED_TIER_COEFICIENT

        score += mon.hp / mon.maxhp * self.HP_FRACTION_COEFICIENT
        score -= opponent.hp/opponent.maxhp * self.HP_FRACTION_COEFICIENT

        return score
    
    def _should_switch_out(self, state):
        active = state.self.active
        opponent = state.opponent.active
        # If there is a decent switch in...
        if active.hp < 1:
            return True
        if len([m for m in state.self.reserve.values() if self._estimate_matchup(m, opponent) > 0]) > 0:
            # ...and a 'good' reason to switch out
            if active.defense_boost <= -3 or active.special_defense_boost <= -3:
                return True
            if (
                active.attack_boost <= -3
                and active.attack >= active.special_attack
            ):
                return True
            if (
                active.special_attack <= -3
                and active.attack <= active.special_attack
            ):
                return True
            if (
                self._estimate_matchup(active, opponent)< self.SWITCH_OUT_MATCHUP_THRESHOLD
            ):
                return True
        return False
    
    def choose_move(self, state):
        # Main mons shortcuts
        active = state.self.active
        opponent = state.opponent.active
        battle_moves = [x for x in state.get_self_options(False) if x[:7] != 'switch ']
        switch_moves = [x for x in state.get_self_options(False) if x[:7] == 'switch ']
#         print(switch_moves)
#         print(active)
#         print([x.hp for x in state.self.reserve.values()])
        if opponent.hp < 1 and active.hp >= 1:
            return 'splash'
        # Rough estimation of damage ratio
        physical_ratio = active.attack / active.defense
        special_ratio = active.special_attack / active.special_defense

        if len(battle_moves) > 0 and (
            not self._should_switch_out(state) or not len(switch_moves) > 0
        ):
            n_remaining_mons = len(
                [m for m in state.self.reserve.values() if m.hp < 1]
            )
            n_opp_remaining_mons = n_remaining_mons

            # Entry hazard...
            for move in battle_moves:
                # ...setup
                if (
                    n_opp_remaining_mons >= 3
                    and move in self.entry_hazards
                    and state.opponent.side_conditions[self.hazard_dict[move]] > 0
                ):
                    return move

                # ...removal
                elif (
                    any(v > 0 for v in state.self.side_conditions.values())
                    and move in self.ANTI_HAZARDS_MOVES
                    and n_remaining_mons >= 2
                ):
                    return move
            damages = []
            for move in battle_moves:
                damage = calculate_damage(state, 'self', move, 'splash', calc_type='average')
                if damage:
                    damages.append(damage[0][0])
                else:
                    damages.append(0)
            if max(damages) > 0 and opponent.hp == 1 or max(damages) > 1:
                return battle_moves[np.argmax(damages)]
        if switch_moves:
            switch_options = []
            for switch in switch_moves:
                mon = switch[7:]
                switch_options.append(self._estimate_matchup(state.self.reserve[mon], opponent))
            return switch_moves[np.argmax(switch_options)]
        
        return state.get_self_options(False)[np.random.randint(len(state.get_self_options(False)))]
    
class PokeBandit():
    def __init__(self, mutator, shp = None):
        self.mutator = mutator
        self.choices = self.mutator.state.get_self_options(False)
        self.n_arms = len(self.choices)
        self.q = np.zeros(self.n_arms)
        self.n = np.zeros(self.n_arms)
        self.shp = shp
        self.state_log = []
        self.move_log = []
        self.instruction_log = []
        self.SPEED_TIER_COEFICIENT = 0.1
        self.HP_FRACTION_COEFICIENT = 0.4
        self.turn_avg = np.zeros(self.n_arms)
    
    def select_arm(self):
        new_arm = np.where(self.n == 0)[0]
        if len(new_arm) > 0:
            return new_arm[0]
        
        ucb1 = self.q/self.n + np.sqrt(2 * np.log(np.sum(self.n)) / self.n)
        return np.argmax(ucb1)
    
    def select_opp_arm(self):
        
        new_arm = np.where(self.n == 0)[0]
        if len(new_arm) > 0:
            return new_arm[0]
        
#         if np.random.random() < 0.1:
#             return shp_select      
        ucb1 = self.q/self.n + np.sqrt(2 * np.log(np.sum(self.n)) / self.n)
        return np.argmax(ucb1)   
    
    
    def create_opponent(self):
        opp_mutator = deepcopy(self.mutator)
        #opp_mutator = pickle.loads(pickle.dumps(self.mutator))
        opp_mutator.state = self.flip_state(opp_mutator.state)
        self.opponent = PokeBandit(mutator=opp_mutator, shp=self.shp)
    
    def flip_state(self, state):
        state_flipped = deepcopy(state)
        tmp = state_flipped.self
        state_flipped.self = state_flipped.opponent
        state_flipped.opponent = tmp
        return state_flipped
    
    def playout_policy(self, state):
        if np.random.random() < 0.2:
            return self.shp.choose_move(state)
        return self.legal_move(state)
    
    def opponent_bandit_policy(self, state):
        if np.random.random() < 0.8:
            return self.shp.choose_move(state)
        return self.legal_move(state)
    
    def estimate_matchup(self, mon, opponent):
        score = self.custom_damage_multiplier(mon, opponent)
        score -= self.custom_damage_multiplier(opponent, mon)
        if mon.speed > opponent.speed:
            score += self.SPEED_TIER_COEFICIENT
        elif opponent.speed > mon.speed:
            score -= self.SPEED_TIER_COEFICIENT

        score += mon.hp / mon.maxhp * self.HP_FRACTION_COEFICIENT
        score -= opponent.hp/opponent.maxhp * self.HP_FRACTION_COEFICIENT

        return score
    
    def custom_damage_multiplier(self, mon, opponent):
        result = []
        for my_type in mon.types:
            mod = 1.0
            for your_type in opponent.types:
                mod *= TYPE_CHART[my_type.upper()][your_type.upper()]
            result.append(mod)
        return max(result)
    
    def legal_move(self, state):
        # Main mons shortcuts
        active = state.self.active
        opponent = state.opponent.active
        battle_moves = [x for x in state.get_self_options(False) if x[:7] != 'switch ']
        switch_moves = [x for x in state.get_self_options(False) if x[:7] == 'switch ']
        if opponent.hp < 1 and active.hp >= 1:
            return 'splash'
        if active.hp < 1:
            switch_options = []
            for switch in switch_moves:
                mon = switch[7:]
                switch_options.append(self.estimate_matchup(state.self.reserve[mon], opponent))
            return switch_moves[np.argmax(switch_options)]
        
        return state.get_self_options(False)[np.random.randint(len(state.get_self_options(False)))]
    
    
    def move(self, new_battle, my_move, opp_move):
        transpose_instructions = get_all_state_instructions(new_battle, my_move,opp_move)
        percentage_list = [x.percentage for x in transpose_instructions]
        instruction_list = [x.instructions for x in transpose_instructions]
#         print(transpose_instructions)
        instruction = random.choices(instruction_list, weights=percentage_list, k=1)[0]
#         print('')
#         print(instruction)
        new_battle.apply(instruction)
        self.instruction_log[-1].append(instruction)
        
        if 'switch ' !=  my_move[:7]:
            for x in new_battle.state.self.active.moves:
                if x['id'] == my_move:
                    if new_battle.state.opponent.active.ability != 'Pressure':
                        x['current_pp'] -= 1
                    else:
                        x['current_pp'] -= 2
                    if x['current_pp'] <= 0:
                        x['disabled'] = True
        else:
            for x in new_battle.state.self.reserve.values():
                x.attack_boost = 0
                x.defense_boost = 0
                x.special_attack_boost = 0
                x.special_defense_boost = 0
                x.speed_boost = 0
                x.evasion_boost = 0
                x.accuracy_boost = 0
                if hasattr(x, 'volatileStatus'):
                    x.volatileStatus = set()

        if 'switch ' !=  opp_move[:7]:
            for x in new_battle.state.opponent.active.moves:
                if x['id'] == opp_move:
                    if new_battle.state.self.active.ability != 'Pressure':
                        x['current_pp'] -= 1
                    else:
                        x['current_pp'] -= 2
                    if x['current_pp'] <= 0:
                        x['disabled'] = True

        else:
            for x in new_battle.state.self.reserve.values():
                x.attack_boost = 0
                x.defense_boost = 0
                x.special_attack_boost = 0
                x.special_defense_boost = 0
                x.speed_boost = 0                
                x.evasion_boost = 0
                x.accuracy_boost = 0  
                if hasattr(x, 'volatileStatus'):
                    x.volatileStatus = set()
                    
        return new_battle
    
    def pull_arm(self, selected_arm, opponent_arm):
        new_battle = deepcopy(self.mutator)
        #new_battle = pickle.loads(pickle.dumps(self.mutator))
        new_state = new_battle.state
        self.state_log[-1].append(deepcopy(new_battle.state))
        flipped_state = self.flip_state(new_state)
        my_move = self.choices[selected_arm]        
        opp_move = self.opponent.choices[opponent_arm]
        self.move_log[-1].append((my_move, opp_move))

#         print(new_battle.state.self.active)
#         print('')
#         print(new_battle.state.opponent.active)     
#         print('')
#         print(new_battle.state.field, new_battle.state.terrain_count)
#         print('')
#         print(my_move, opp_move)   
        new_battle = self.move(new_battle, my_move, opp_move)
#         print('-----------------------------------------------') 
        turns = 1
        while new_battle.state.battle_is_finished() == False:
#             print(new_battle.state.self.active)
#             print('')
#             print(new_battle.state.opponent.active)     
#             print('')
#             print(new_battle.state.field, new_battle.state.terrain_count, new_battle.state.weather_count)
#             print('')

            flipped_state = self.flip_state(new_battle.state)
            self.state_log[-1].append(deepcopy(new_battle.state))
            my_move = self.playout_policy(new_battle.state)
            opp_move = self.opponent.playout_policy(flipped_state)
            self.move_log[-1].append((my_move, opp_move))
            new_battle = self.move(new_battle, my_move, opp_move)
            turns += 1
#             if len(self.state_log[-1]) > 10:
#                 print('triple')
#                 print(self.state_log[-1][0].self.active)
#                 print(self.state_log[-1][5].self.active)
#                 print(self.state_log[-1][10].self.active)
#             print(my_move, opp_move)   
#             print('-----------------------------------------------') 
        reward = new_battle.state.battle_is_finished()
        if reward == -1:
            reward = 0
        return reward, turns
    
    def play_game(self, simulations_number=None, total_simulation_seconds=None):
        """
        Parameters
        ----------
        simulations_number : int
            number of simulations performed to get the best action
        total_simulation_seconds : float
            Amount of time the algorithm has to run. Specified in seconds
        Returns
        -------
        """
        
        self.create_opponent()
        if simulations_number is None :
            assert(total_simulation_seconds is not None)
            end_time = time.time() + total_simulation_seconds
            opp_state = self.flip_state(self.mutator.state)
            opp_options = opp_state.get_self_options(False)
            shp_select = opp_options.index(self.shp.choose_move(opp_state))
            while time.time() < end_time:
                chosen_arm = self.select_arm()
                opponent_arm = self.opponent.select_opp_arm(shp_select)
                reward, turns = self.pull_arm(chosen_arm, opponent_arm)
                self.n[chosen_arm] += 1
                self.q[chosen_arm] += reward
                self.opponent.n[opponent_arm] += 1
                self.opponent.q[opponent_arm] += 1 - reward
                self.turn_avg[chosen_arm] += turns
                
                
                if np.sum(self.n) > 200:
                    if all(self.q/self.n > 0.9):
                        print('ez')
                        break
                    error_bound = (self.q * (1 - self.q / self.n) ** 2 + 
                                   (self.n-self.q) * (self.q/self.n) ** 2)/self.n/np.sqrt(self.n)

                    upper_bound = self.q/self.n + 1.95*error_bound
                    lower_bound = self.q/self.n - 1.95*error_bound
                    if sum(max(lower_bound) < upper_bound) == 1:
                        print('confident')
                        break


        else :
#             opp_state = self.flip_state(self.mutator.state)
#             opp_options = opp_state.get_self_options(False)
#             shp_select = opp_options.index(self.shp.choose_move(opp_state))
            for _ in range(0, simulations_number):    
                self.move_log.append([])
                self.state_log.append([])
                self.instruction_log.append([])
                chosen_arm = self.select_arm()
                opponent_arm = self.opponent.select_opp_arm()
                reward, turns = self.pull_arm(chosen_arm, opponent_arm)
                self.n[chosen_arm] += 1
                self.q[chosen_arm] += reward
                self.opponent.n[opponent_arm] += 1
                self.opponent.q[opponent_arm] += 1 - reward
                self.turn_avg[chosen_arm] += turns
                
                if np.sum(self.n) > 200:
                    if all(self.q/self.n > 0.9):
                        print('ez')
                        break
                    error_bound = (self.q * (1 - self.q / self.n) ** 2 + 
                                   (self.n-self.q) * (self.q/self.n) ** 2)/self.n/np.sqrt(self.n)

                    upper_bound = self.q/self.n + 1.95*error_bound
                    lower_bound = self.q/self.n - 1.95*error_bound
                    if sum(max(lower_bound) < upper_bound) == 1:
                        print('confident')
                        break


                
    def best_action(self, simulations_number=None, total_simulation_seconds=None):
        self.play_game(simulations_number, total_simulation_seconds)
        self.turn_avg /= self.n
        if all(self.q/self.n > 0.9) and np.random.random() < 0.3:
            return self.shp.choose_move(self.mutator.state)
        return self.choices[np.argmax(self.q/self.n)]

In [23]:
with open('mut_ex.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(mut_ex_copy, f)

In [24]:
with open('mut_ex.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    mut_ex_copy = pickle.load(f)

In [84]:
mut_ex_copy = deepcopy(mut_ex)

In [25]:
config.damage_calc_type = 'average'

In [27]:
import sys, importlib
importlib.reload(sys.modules['showdown.engine.find_state_instructions'])
importlib.reload(sys.modules['showdown.engine.damage_calculator'])
importlib.reload(sys.modules['showdown.engine.instruction_generator'])
importlib.reload(sys.modules['showdown.engine.objects'])
importlib.reload(sys.modules['showdown.engine.special_effects.abilities.on_switch_in'])
#importlib.reload(sys.modules['data'])
from showdown.engine.find_state_instructions import get_all_state_instructions, get_state_instructions_from_move
from showdown.engine.damage_calculator import calculate_damage
from showdown.engine.objects import Pokemon, State, StateMutator, Side
from showdown.engine.special_effects.abilities.on_switch_in import ability_on_switch_in

In [42]:
mut_ex_copy.state.self

{'active': {'id': 'clefable', 'level': 100, 'types': ['fairy'], 'hp': 394, 'maxhp': 394, 'ability': 'magicguard', 'item': 'leftovers', 'attack': 158, 'defense': 203, 'special-attack': 226, 'special-defense': 276, 'speed': 164, 'nature': 'serious', 'evs': (85, 85, 85, 85, 85, 85), 'attack_boost': 0, 'defense_boost': 0, 'special_attack_boost': 0, 'special_defense_boost': 0, 'speed_boost': 0, 'accuracy_boost': 0, 'evasion_boost': 0, 'status': None, 'volatileStatus': [], 'moves': [{'id': 'moonblast', 'disabled': False, 'current_pp': 15}, {'id': 'knockoff', 'disabled': False, 'current_pp': 20}, {'id': 'softboiled', 'disabled': False, 'current_pp': 10}, {'id': 'thunderwave', 'disabled': False, 'current_pp': 20}], 'part_trapped_counter': 0, 'substitute_hp': 0}, 'reserve': {'zapdos': {'id': 'zapdos', 'level': 100, 'types': ['electric', 'flying'], 'hp': 383, 'maxhp': 383, 'ability': 'static', 'item': 'heavydutyboots', 'attack': 166, 'defense': 293, 'special-attack': 286, 'special-defense': 216,

In [39]:
with open('team2.txt', 'r') as f:
    for line in f:
        print(line)

Hydreigon @ Life Orb

Ability: Levitate

EVs: 4 Atk / 252 SpA / 252 Spe

Hasty Nature

- Draco Meteor

- Superpower

- Flash Cannon

- Roost



Zapdos @ Heavy-Duty Boots

Ability: Static

EVs: 248 HP / 244 Def / 16 Spe

Bold Nature

IVs: 0 Atk

- Volt Switch

- Hurricane

- Roost

- Defog



Clefable @ Leftovers

Ability: Magic Guard

EVs: 252 HP / 84 Def / 140 SpD / 32 Spe

Calm Nature

- Moonblast

- Knock Off

- Soft-Boiled

- Thunder Wave



Landorus-Therian (M) @ Leftovers

Ability: Intimidate

EVs: 252 HP / 4 Atk / 252 Def

Impish Nature

IVs: 30 Spe

- Stealth Rock

- Earthquake

- Toxic

- U-turn



Melmetal @ Assault Vest

Ability: Iron Fist

EVs: 128 HP / 116 Atk / 252 SpD / 12 Spe

Careful Nature

- Double Iron Bash

- Earthquake

- Ice Punch

- Thunder Punch



Dragapult @ Choice Specs

Ability: Infiltrator

EVs: 252 SpA / 4 SpD / 252 Spe

Timid Nature

- Draco Meteor

- Shadow Ball

- Hex

- U-turn



In [28]:
bandit = PokeBandit(mut_ex_copy, shp = HeuristicsPlayer())
bandit.play_game(simulations_number=1)
#bandit.play_game(total_simulation_seconds=15)

In [37]:
display_battle(bandit.state_log, bandit.move_log, bandit.instruction_log,0)

TURN 1

My side conditions- 

     clefable, leftovers
|-----------------------------------|
        100%, 394/394

Weather:  None

     zapdos, unknown_item
|--------------------------------------|
        100%, 384/384

Opponent side conditions- 

-----------------------------------------------------------------------------------------------------
My move: moonblast
Opp move: roost
Instructions:
[('apply_volatile_status', 'opponent', 'roost'), ('damage', 'opponent', 91.0), ('remove_volatile_status', 'opponent', 'roost')]
-----------------------------------------------------------------------------------------------------

My side conditions- 

     clefable, leftovers
|-----------------------------------|
        100%, 394/394

Weather:  None

     zapdos, unknown_item
|-----------------------------      |
        77%, 293/384

Opponent side conditions- 

-----------------------------------------------------------------------------------------------------
TURN 2

My side conditions- 

-----------------------------------------------------------------------------------------------------
My move: knockoff
Opp move: switch dragapult
Instructions:
[('switch', 'opponent', 'landorustherian', 'dragapult'), ('damage', 'opponent', 88.0), ('change_item', 'opponent', None, 'choicespecs'), ('heal', 'self', 25)]
-----------------------------------------------------------------------------------------------------

My side conditions- 

     clefable, leftovers, att_b: -1
|------------------------           |
        69%, 268/394

Weather:  None

     dragapult, None, eva_b: -1
|-----------------                  |
        46%, 173/380

Opponent side conditions- 

-----------------------------------------------------------------------------------------------------
TURN 50

My side conditions- 

     clefable, leftovers, att_b: -1
|------------------------           |
        69%, 268/394

Weather:  None

     dragapult, None, eva_b: -1
|-----------------                  |
        

Opponent side conditions- 

-----------------------------------------------------------------------------------------------------
TURN 87

My side conditions- 

     clefable, leftovers, att_b: -3, spa_b: -1
|-----------------                  |
        49%, 191/394, tox

Weather:  None

     clefable, None, spa_b: 3, spdef_b: 4
|-----------------------------------|
        100%, 394/394, par

Opponent side conditions- 

-----------------------------------------------------------------------------------------------------
My move: knockoff
Opp move: switch landorustherian
Instructions:
[('unboost', 'opponent', 'special-attack', 3), ('unboost', 'opponent', 'special-defense', 4), ('switch', 'opponent', 'clefable', 'landorustherian'), ('unboost', 'self', 'attack', 1), ('damage', 'opponent', 19.0), ('change_item', 'opponent', None, 'leftovers'), ('heal', 'self', 25)]
-----------------------------------------------------------------------------------------------------

My side conditions- 



IndexError: list index out of range

In [30]:
def display_battle(state_log, move_log, instruction_log, battle_num):
    battle_len = len(state_log[battle_num])
    for i in range(battle_len):
        print('TURN ' + str(i+1))
        print('')
        display_turn(state_log, move_log, instruction_log, battle_num, i)
        print('')
        print('-----------------------------------------------------------------------------------------------------')

In [31]:
def display_turn(state_log, move_log, instruction_log, battle_num, turn_num):
    instructions = instruction_log[battle_num][turn_num]
    moves = move_log[battle_num][turn_num]
    display_side(state_log, move_log, battle_num, turn_num)
    print('')
    print('-----------------------------------------------------------------------------------------------------')
    print('My move: ' + moves[0])
    print('Opp move: ' + moves[1])
    print('Instructions:' )
    print(instructions)
    print('-----------------------------------------------------------------------------------------------------')
    print('')
    display_side(state_log, move_log, battle_num, turn_num + 1)

In [32]:
def display_side(state_log, move_log,battle_num, turn_num):
    turn = state_log[battle_num][turn_num]
    moves = move_log[battle_num][turn_num]
    side_conds = 'My side conditions- '
    for i in turn.self.side_conditions:
        if turn.self.side_conditions[i] != 0:
            side_conds += i +': ' + str(turn.self.side_conditions[i])
    print(side_conds)
    print('')
    display_mon(turn.self.active)
    print('')
    print('Weather: ', turn.weather)
    print('')
    display_mon(turn.opponent.active)
    print('')
    opp_side_conds = 'Opponent side conditions- '
    for i in turn.opponent.side_conditions:
        if turn.opponent.side_conditions[i] != 0:
            opp_side_conds += i +': ' + str(turn.opponent.side_conditions[i])
    print(opp_side_conds)

In [33]:
def display_mon(mon):
    info = '     ' + mon.id + ', ' + str(mon.item)
    if mon.attack_boost != 0:
        info += ', att_b: ' + str(mon.attack_boost)
    if mon.defense_boost != 0:
        info += ', def_b: ' + str(mon.defense_boost)
    if mon.special_attack_boost != 0:
        info += ', spa_b: ' + str(mon.special_attack_boost)
    if mon.special_defense_boost != 0:
        info += ', spdef_b: ' + str(mon.special_defense_boost)
    if mon.speed_boost != 0:
        info += ', spd_b: ' + str(mon.speed_boost)
    if mon.accuracy_boost != 0:
        info += ', acc_b: ' + str(mon.accuracy_boost)
    if mon.evasion_boost != 0:
        info += ', eva_b: ' + str(mon.evasion_boost)
    print(info)
    do_health(mon.hp, mon.maxhp, mon.status, 35)

In [35]:
def do_health(health, maxHealth, status,healthDashes):
    dashConvert = int(maxHealth/healthDashes)            # Get the number to divide by to convert health to dashes (being 10)
    currentDashes = int(health/dashConvert)              # Convert health to dash count: 80/10 => 8 dashes
    remainingHealth = healthDashes - currentDashes       # Get the health remaining to fill as space => 12 spaces

    healthDisplay = '-' * currentDashes                  # Convert 8 to 8 dashes as a string:   "--------"
    remainingDisplay = ' ' * remainingHealth             # Convert 12 to 12 spaces as a string: "            "
    percent = str(int(np.ceil(health/maxHealth*100) ))+ "%"     # Get the percent as a whole number:   40%

    print("|" + healthDisplay + remainingDisplay + "|")  # Print out textbased healthbar
    if status == None:
        print("        " + percent + ', ' + str(int(health)) + '/' + str(maxHealth))   
    else:
        print("        " + percent + ', ' + str(int(health)) + '/' + str(maxHealth) + ', '+ status)  

In [54]:
print(len(bandit.state_log[0]))

71


In [55]:
print(len(bandit.move_log[0]))

71


In [49]:
bandit.move_log[0]

[('stealthrock', 'moonblast'),
 ('switch zapdos', 'switch heatran'),
 ('switch melmetal', 'taunt'),
 ('switch dragapult', 'switch slowking'),
 ('switch zapdos', 'switch clefable'),
 ('switch melmetal', 'moonblast'),
 ('doubleironbash', 'moonblast'),
 ('splash', 'switch heatran'),
 ('switch zapdos', 'magmastorm'),
 ('switch dragapult', 'switch zeraora'),
 ('dracometeor', 'switch corviknight'),
 ('dracometeor', 'uturn'),
 ('dracometeor', 'switch zeraora'),
 ('dracometeor', 'switch heatran'),
 ('switch landorustherian', 'switch corviknight'),
 ('switch clefable', 'uturn'),
 ('switch zapdos', 'uturn'),
 ('voltswitch', 'switch corviknight'),
 ('splash', 'switch landorustherian'),
 ('dracometeor', 'uturn'),
 ('switch clefable', 'voltswitch'),
 ('moonblast', 'uturn'),
 ('switch dragapult', 'switch heatran'),
 ('shadowball', 'earthpower'),
 ('shadowball', 'switch slowking'),
 ('switch zapdos', 'switch heatran'),
 ('voltswitch', 'magmastorm'),
 ('moonblast', 'taunt'),
 ('knockoff', 'earthpower'

In [14]:
%load_ext line_profiler

In [15]:
bandit = PokeBandit(mut_ex_copy, shp = HeuristicsPlayer())

In [20]:
%lprun -f get_state_instructions_from_move bandit.play_game(simulations_number=5)

In [38]:
from showdown.engine.instruction_generator import get_end_of_turn_instructions

In [34]:
np.var([1, 1, 1, 0, 0, 0, 0])

0.24489795918367346

In [40]:
(3 * (1-3/7)**2 + 4*(3/7)**2)/7

0.24489795918367346

In [46]:
(1 - bandit.q / bandit.n) ** 2

array([0.311, 0.311, 0.299, 0.298, 0.323, 0.25 , 0.276, 0.276, 0.366])

In [47]:
bandit.q

array([23., 23., 24., 25., 22., 34., 28., 28., 17.])

In [54]:
error = ((bandit.q * (1 - bandit.q / bandit.n) ** 2 + (bandit.n-bandit.q) * (bandit.q/bandit.n) ** 2)/bandit.n) / np.sqrt(bandit.n)

In [75]:
upper_bound = bandit.q/bandit.n + 1.6*error
lower_bound = bandit.q/bandit.n - 1.6*error
sum(max(lower_bound) < upper_bound)

9

In [14]:
mut_ex.state == deepcopy(mut_ex.state)

False

9

In [63]:
(bandit.q/bandit.n + 1.6*error) < [min_max

array([ True,  True,  True,  True,  True,  True,  True,  True,  True])

In [29]:
print(np.sum(bandit.n))
print(bandit.q)
print(bandit.n)
print("Win %: ", bandit.q/bandit.n)
print("Play % ", bandit.n/np.sum(bandit.n))
print('Formula: ', bandit.q/bandit.n + bandit.n/np.sum(bandit.n))
print('Turns: ', bandit.turn_avg)
print(bandit.choices)

492.0
[23. 23. 24. 25. 22. 34. 28. 28. 17.]
[52. 52. 53. 55. 51. 68. 59. 59. 43.]
Win %:  [0.442 0.442 0.453 0.455 0.431 0.5   0.475 0.475 0.395]
Play %  [0.106 0.106 0.108 0.112 0.104 0.138 0.12  0.12  0.087]
Formula:  [0.548 0.548 0.561 0.566 0.535 0.638 0.594 0.594 0.483]
Turns:  [39.692 36.788 41.66  39.236 37.294 41.088 40.593 41.322 45.767]
['earthquake', 'explosion', 'stealthrock', 'swordsdance', 'switch garchomp', 'switch tapukoko', 'switch azumarill', 'switch dragonite', 'switch hawlucha']


In [30]:
print(np.sum(bandit.opponent.n))
print(bandit.opponent.q)
print(bandit.opponent.n)
print("Win %: ", bandit.opponent.q/bandit.opponent.n)
print("Play % ", bandit.opponent.n/np.sum(bandit.opponent.n))
print('Formula: ', bandit.opponent.q/bandit.opponent.n + bandit.opponent.n/np.sum(bandit.opponent.n))
print(bandit.opponent.choices)

492.0
[32. 33. 50. 21. 20. 29. 28. 41. 14.]
[58. 58. 80. 44. 42. 54. 53. 69. 34.]
Win %:  [0.552 0.569 0.625 0.477 0.476 0.537 0.528 0.594 0.412]
Play %  [0.118 0.118 0.163 0.089 0.085 0.11  0.108 0.14  0.069]
Formula:  [0.67  0.687 0.788 0.567 0.562 0.647 0.636 0.734 0.481]
['earthquake', 'uturn', 'stealthrock', 'knockoff', 'switch corviknight', 'switch clefable', 'switch slowking', 'switch heatran', 'switch zeraora']


In [ ]:
    def pull_arm(self, selected_arm):
        new_battle = deepcopy(self.mutator)
        new_state = new_battle.state
        self.state_log.append(new_state)
        flipped_state = self.flip_state(new_state)
        my_move = self.choices[self.select_arm()]
        opp_move = self.opponent_bandit_policy(flipped_state)
#         print(new_battle.state.self.active)
#         print('')
#         print(new_battle.state.opponent.active)     
#         print('')
#         print(my_move, opp_move)
        self.move_log.append((my_move, opp_move))
        new_battle = self.move(new_battle, my_move, opp_move)
#         print('-----------------------------------------------')    
        while new_battle.state.battle_is_finished() == False:
#             print(new_battle.state.self.active)
#             print('')
#             print(new_battle.state.opponent.active)     
#             print('')
            self.state_log.append(new_battle.state)
            flipped_state = self.flip_state(new_battle.state)
            my_move = self.playout_policy(new_battle.state)
            opp_move = self.playout_policy(flipped_state)
#            print(my_move, opp_move)   
            self.move_log.append((my_move, opp_move))
            new_battle = self.move(new_battle, my_move, opp_move)
#            print('-----------------------------------------------') 
        
        reward = new_battle.state.battle_is_finished()
        if reward == -1:
            reward = 0
        return reward

In [ ]:
for move_name, move_info in all_moves.items():
    move_info['id'] = move_name
    move_info['category'] = move_info['category'].lower()
    move_info['type'] = move_info['type'].lower()
    if 'boosts' in move_info:
        if 'atk' in move_info['boosts']:
            move_info['boosts']['attack'] = move_info['boosts']['atk']
            move_info['boosts'].pop('atk', None)
        if 'def' in move_info['boosts']:
            move_info['boosts']['defense'] = move_info['boosts']['def']
            move_info['boosts'].pop('def', None)
        if 'spa' in move_info['boosts']:
            move_info['boosts']['special-attack'] = move_info['boosts']['spa']
            move_info['boosts'].pop('spa', None)
        if 'spd' in move_info['boosts']:
            move_info['boosts']['special-defense'] = move_info['boosts']['spd']
            move_info['boosts'].pop('spd', None)
        if 'spe' in move_info['boosts']:
            move_info['boosts']['speed'] = move_info['boosts']['spe']
            move_info['boosts'].pop('spe', None)

    if 'self' in move_info and 'boosts' in move_info['self']:
        if 'atk' in move_info['self']['boosts']:
            move_info['self']['boosts']['attack'] = move_info['self']['boosts']['atk']
            move_info['self']['boosts'].pop('atk', None)
        if 'def' in move_info['self']['boosts']:
            move_info['self']['boosts']['defense'] = move_info['self']['boosts']['def']
            move_info['self']['boosts'].pop('def', None)
        if 'spa' in move_info['self']['boosts']:
            move_info['self']['boosts']['special-attack'] = move_info['self']['boosts']['spa']
            move_info['self']['boosts'].pop('spa', None)
        if 'spd' in move_info['self']['boosts']:
            move_info['self']['boosts']['special-defense'] = move_info['self']['boosts']['spd']
            move_info['self']['boosts'].pop('spd', None)
        if 'spe' in move_info['self']['boosts']:
            move_info['self']['boosts']['speed'] = move_info['self']['boosts']['spe']
            move_info['self']['boosts'].pop('spe', None)
    if 'secondary' not in move_info or move_info['secondary'] == None or move_info['secondary'] == False:
        move_info['secondary'] = False
    
    elif 'secondary' in move_info and 'boosts' in move_info['secondary']:
        if 'atk' in move_info['secondary']['boosts']:
            move_info['secondary']['boosts']['attack'] = move_info['secondary']['boosts']['atk']
            move_info['secondary']['boosts'].pop('atk', None)
        if 'def' in move_info['secondary']['boosts']:
            move_info['secondary']['boosts']['defense'] = move_info['secondary']['boosts']['def']
            move_info['secondary']['boosts'].pop('def', None)
        if 'spa' in move_info['secondary']['boosts']:
            move_info['secondary']['boosts']['special-attack'] = move_info['secondary']['boosts']['spa']
            move_info['secondary']['boosts'].pop('spa', None)
        if 'spd' in move_info['secondary']['boosts']:
            move_info['secondary']['boosts']['special-defense'] = move_info['secondary']['boosts']['spd']
            move_info['secondary']['boosts'].pop('spd', None)
        if 'spe' in move_info['secondary']['boosts']:
            move_info['secondary']['boosts']['speed'] = move_info['secondary']['boosts']['spe']
            move_info['secondary']['boosts'].pop('spe', None)
    elif 'secondary' in move_info and 'self' in move_info['secondary'] and 'boosts' in move_info['secondary']['self']:
        if 'atk' in move_info['secondary']['self']['boosts']:
            move_info['secondary']['self']['boosts']['attack'] = move_info['secondary']['self']['boosts']['atk']
            move_info['secondary']['self']['boosts'].pop('atk', None)
        if 'def' in move_info['secondary']['self']['boosts']:
            move_info['secondary']['self']['boosts']['defense'] = move_info['secondary']['self']['boosts']['def']
            move_info['secondary']['self']['boosts'].pop('def', None)
        if 'spa' in move_info['secondary']['self']['boosts']:
            move_info['secondary']['self']['boosts']['special-attack'] = move_info['secondary']['self']['boosts']['spa']
            move_info['secondary']['self']['boosts'].pop('spa', None)
        if 'spd' in move_info['secondary']['self']['boosts']:
            move_info['secondary']['self']['boosts']['special-defense'] = move_info['secondary']['self']['boosts']['spd']
            move_info['secondary']['self']['boosts'].pop('spd', None)
        if 'spe' in move_info['secondary']['self']['boosts']:
            move_info['secondary']['self']['boosts']['speed'] = move_info['secondary']['self']['boosts']['spe']
            move_info['secondary']['self']['boosts'].pop('spe', None)